<a href="https://colab.research.google.com/github/lyuoveta/git_test_model_1000/blob/master/Llama%20/Llama_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 5.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 42.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 9.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-bejuhvim/overlay/local/bin
    changing mode of /tmp/pip-build-env-bejuhvim/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-bejuhvim/overlay/l

In [7]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [8]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [9]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [10]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [11]:

from google.colab import files


pd= files.upload()

Saving tagged_data_llama_256_1.csv to tagged_data_llama_256_1.csv


In [12]:
import pandas as pd
import random
df = pd.read_csv('tagged_data_llama_256_1.csv')


In [13]:
df = df[['reviews', 'Correct review ', 'Incorrect review ']]
df.head()

,reviews,Correct review,Incorrect review
0,Not bad but could be better,1,0
1,Taking biotin 5000mcg from October first week ...,1,0
2,My nails grew in noticeably thicker and strong...,1,0
3,I took biotin for 1 week. I started having mu...,1,0
4,I used to have very thin nails that bent when ...,1,0


In [14]:
df_incorrect = df.loc[df['Incorrect review '] == 1]['reviews'].reset_index(drop=True)
df_correct = df.loc[df['Correct review '] == 1]['reviews'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'This stuff is Great! I take 2 a day and my nails grow faster and they are thicker. I Love this stuff and my hair grows faster and is not falling out. It almost looks like some of my grey hair is getting color back in it.'

In [15]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'Would there possibly be a skin rash side effect?'

In [ ]:
#Step 6: Generating the Response

In [19]:
for _ in range(5):
    prompt = f'''Examples of drug review ratings:
    1. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    2. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    3. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    4. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    5. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1

    Please provide the correctness only of the following review by entering 0 for incorrect or 1 for correct:
    "{df['reviews'][random.choice(range(len(df)))]}"
    '''
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.

    USER: {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)
    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    matches = re.findall(r'ASSISTANT:.*?([01])', response_text, re.DOTALL)

    print(matches)

Llama.generate: prefix-match hit


{'id': 'cmpl-93ce4c6d-21e7-4920-8d4a-47f20681a269', 'object': 'text_completion', 'created': 1701614287, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Examples of drug review ratings:\n    1. Be very careful taking Vit.D supplement.  It was found that while my Vit.D levels were increasing so were my CALCIUM levels.  Have been advised by my doctor to take 1 Vit.D every other day but feeling so bad I am not sure of what to do now.\n     ANSWER: 1\n    2. I was looking for any negative side effects, as I am required to take 100,000 iu a week.\n     ANSWER: 0\n    3. In 2 years CHO went from over 250 to 180.\n     ANSWER: 1\n    4. peterkings2014\n     ANSWER: 0\n    5. I found a new (?) use for colloidal silver.  I gave some to my 10 lb dog and he stoppe

Llama.generate: prefix-match hit


{'id': 'cmpl-8b21e680-f268-4ee0-87ff-3f5738faf41e', 'object': 'text_completion', 'created': 1701614338, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Examples of drug review ratings:\n    1. I use to treat MS and high cholesterol, and I feel better when taking it.  My symptons are lessened like muscle spams, memory and overall aches and pain.\n     ANSWER: 1\n    2. Whole Foods Market does not sell it for a reason. It is not sustainable.\n     ANSWER: 0\n    3. My sister has had an adverse reaction to antibiotics and has loss hair on numerous occasions. In some cases almost to near baldness. Biotin has always helped in a big way.\n     ANSWER: 1\n    4. first time\n     ANSWER: 0\n    5. Been taking the suggested amount for almost 12 weeks now and Tb

Llama.generate: prefix-match hit


{'id': 'cmpl-0ce5c39e-7e64-4633-8cf7-b2f86bcde20d', 'object': 'text_completion', 'created': 1701614379, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Examples of drug review ratings:\n    1. My Dr. told me to try it.I did and it worked. It used to be obtainable on the National Health,but now they´ve taken it off.\n     ANSWER: 1\n    2. Vitamin D is a hormone. This article states it it is a vitamin. It is referred to as a vitamin because they body cannot make it on its own, like all other hormones.\n     ANSWER: 0\n    3. When I’ve been around ill people or feel a slight sore throat I put a dropper full under my tongue and a few drops into each nostril and it makes me feel much better. I also use it in my Nebulizer to prevent or quiet flu or respirat

Llama.generate: prefix-match hit


{'id': 'cmpl-51d42efd-b4e8-4193-b834-1c785aab4dae', 'object': 'text_completion', 'created': 1701614421, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Examples of drug review ratings:\n    1. Somewhat effective.  Has an after wake up feeling of unfresh\n     ANSWER: 1\n    2. Take Part of my morning regimen. Prescribed/recommended by my family physician.\n     ANSWER: 0\n    3. Our family has used CS for years; both for us and our pets.  Strep throat, wounds, a rescue dog with MRSA and a horrible mouth with infected gums, and much more.  It\'s worked every time.  We never use antibiotics anymore.  CS works fast and completely.\n     ANSWER: 1\n    4. g mail\n     ANSWER: 0\n    5. This supplement made me bloat at 1500mg. I also had tingling sensation 

Llama.generate: prefix-match hit


{'id': 'cmpl-c70da035-e436-4fb4-8892-99e4dbf81eb9', 'object': 'text_completion', 'created': 1701614519, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM: You are a helpful, respectful and honest assistant. Always answer accurately.\n\n    USER: Examples of drug review ratings:\n    1. After taking krill oil all sciatica problems gone and cholestral levels reduced.\n     ANSWER: 1\n    2. one ear still block\n     ANSWER: 0\n    3. Berberine with cinnamon has helped with morning glucose numbers. Was wondering if safe long term\n     ANSWER: 1\n    4. I\'m taking 2,400 IU of vitamin D3 daily,Is that to much?\n     ANSWER: 0\n    5. Using 15 mg. of 5-MTHF from Extrafolate-S.  Only been using it for about two 1/2 weeks.  DNA tests showed I was low in producing folate.  I can\'t tell any difference so far.  It is supposed to support my psychiatric 

In [20]:
!pip install scikit-learn

true_labels = [0,1,1,1,1,1,1,1,0,1]
predicted_labels = [1,0,1,0,1,1,1,1,0,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")


Точность модели: 70.00%
